# Wind Energy


In [ ]:
import os
import sys

import matplotlib.pyplot as plt
import xarray as xr

try:
    import salientsdk as sk
except ModuleNotFoundError as e:
    if os.path.exists("../salientsdk"):
        sys.path.append(os.path.abspath(".."))
        import salientsdk as sk
    else:
        raise ModuleNotFoundError("Install salient SDK with: pip install salientsdk")


# These are the variables we need to convert wind to power:
vars = "wspd,wspd100,temp"
# Temporal resolution of the time series:
freq = "hourly"
# Analyze one year of data:
(start_date, end_date) = ("2022-01-01", "2022-12-31")
# When we want to plot a timeseries, focus on a specific month:
plt_time = slice("2022-02-01", "2022-02-28")


sk.set_file_destination("wind_example")
sk.login("SALIENT_USERNAME", "SALIENT_PASSWORD")

## Get Source Data

To start, call the Salient API to get an hourly forecast of wind-relevant meteorological varibles. Then get historical observed values for comparison.


### Set geographic bounds

The Salient SDK uses a `Location` object to specify the geographic bounds of a request. Let's analyze wind energy at 3 sites. Escalade (Texas) features modern high-capacity wind turbines on tall towers. Twin Groves (Illinois) and Windy Point (Oregon) have older moderate-capacity turbines on shorter towers.


In [ ]:
if True:  # Analyze wind at 3 locations with a location_file
    loc_file = sk.upload_location_file(
        lats=[33.735371, 40.445091, 45.710293],
        lons=[-99.646866, -88.696297, -120.848785],
        names=["Escalade", "Twin Groves I", "Windy Point IIa"],
        hub_height=[119, 80, 80],  # meters
        elevation=[444, 260, 582],  # meters
        rated_capacity=[5.6, 1.65, 2.3],  # Megawatts
        turbine_count=[65, 121, 24],
        turbine_model=["V162-5.6", "V82-1.65", "SWT-2.3-93"],
        turbine_manufacturer=["Vestas", "Vestas", "Siemens"],
        year_online=[2022, 2007, 2009],
        geoname="wind_example",
        force=False,
    )
    loc = sk.Location(location_file=loc_file)

    def add_turbines(ds, loc, force=False):
        """Add elevation, hub height, and turbine model data to the dataset."""
        return sk.merge_location_data(ds, loc_file)

    def plot_timeseries(ds, timeslice=plt_time):
        """Plot the data vs time."""
        if timeslice is not None:
            ds = ds.sel(time=timeslice)

        if "ensemble" in ds.coords:
            plt_h = ds.plot(
                x="time",
                hue="ensemble",
                col="location",
                size=5,
                aspect=1,
                add_legend=False,
                color="#80808040",
            )
        else:
            # Use this for historical data_timeseries
            ds.plot(x="time", hue="location")

else:  # Analyze wind over the Western ERCOT region with a shapefile
    coords = [
        (-103.25, 32.00),
        (-105.50, 32.00),
        (-105.50, 31.00),
        (-104.50, 29.50),
        (-103.25, 29.00),
        (-102.50, 29.75),
        (-101.25, 29.50),
        (-100.50, 29.00),
        (-98.000, 34.00),
        (-100.00, 34.50),
        (-100.00, 35.75),
        (-103.25, 35.75),
    ]

    shape_file = sk.upload_shapefile(coords, "ercot_west", force=False)
    loc = sk.Location(shapefile=shape_file)

    def add_turbines(ds, loc, force=False):
        """Add elevation, hub height, and turbine model data to the dataset."""
        ds["hub_height"] = xr.DataArray(95)
        ds["turbine_model"] = xr.DataArray("SWT-2.3-93")

        elev = xr.load_dataset(sk.geo(loc, variables="elevation", force=False, verbose=False))
        elev = elev.interp(lat=ds.lat, lon=ds.lon, method="linear")
        ds = ds.merge(elev)

        return ds

    def plot_timeseries(ds, timeslice=plt_time):
        """Plot the data vs time."""
        if timeslice is not None:
            ds = ds.sel(time=timeslice)

        # get all dimensions of the dataset that are not "time"
        dims = [dim for dim in ds.dims if dim != "time"]
        ds_avg = ds.mean(dim=dims)
        ds_std = ds.std(dim=dims)
        ds_avg.attrs = ds.attrs
        ds_std.attrs = ds.attrs

        fig, ax = plt.subplots()
        ds_avg.plot(ax=ax)
        ax.fill_between(
            ds_avg["time"],
            ds_avg - ds_std,
            ds_avg + ds_std,
            color="gray",
            alpha=0.25,
        )

### Get Historical Observed Data


In [ ]:
file_hst = sk.data_timeseries(
    loc=loc,
    variable=vars,
    field="vals",
    frequency=freq,
    start=start_date,
    end=end_date,
)

# Load all of the history files into a single dataset:
hst = sk.load_multihistory(file_hst, fields=["vals"])

# Add extra columns like elevation and turbine_count to the dataset:
hst = add_turbines(hst, loc)
print(hst)

plot_timeseries(hst["wspd100"])
plt.title(f"{freq} historical actuals");

<xarray.Dataset> Size: 699kB
Dimensions:               (location: 3, time: 8737)
Coordinates:
  * location              (location) object 24B 'Escalade' ... 'Windy Point IIa'
  * time                  (time) datetime64[ns] 70kB 2022-01-01 ... 2022-12-31
    lat                   (location) float64 24B 33.74 40.45 45.71
    lon                   (location) float64 24B -99.65 -88.7 -120.8
Data variables:
    hub_height            (location) int64 24B 119 80 80
    elevation             (location) int64 24B 444 260 582
    rated_capacity        (location) float64 24B 5.6 1.65 2.3
    turbine_count         (location) int64 24B 65 121 24
    turbine_model         (location) object 24B 'V162-5.6' ... 'SWT-2.3-93'
    turbine_manufacturer  (location) object 24B 'Vestas' 'Vestas' 'Siemens'
    year_online           (location) int64 24B 2022 2007 2009
    wspd                  (time, location) float64 210kB 3.592 3.517 ... 1.135
    wspd100               (time, location) float64 210kB 6.842 8.0

### Downscale Forecast

The `downscale` API endpoint converts Salient's temporally granular weekly/monthly/quarterly forecasts into an hourly ensemble of timeseries.


In [ ]:
file_dsc = sk.downscale(
    loc=loc,
    variables=vars,
    date=start_date,
    frequency=freq,
    members=11,
)

dsc = add_turbines(xr.load_dataset(file_dsc), loc)

# Make downscaled forecasts consistent with historical timeseries
if "forecast_day" in dsc:
    dsc = dsc.rename({"forecast_day": "time"})

print(dsc)
plot_timeseries(dsc["wspd100"])

<xarray.Dataset> Size: 4MB
Dimensions:               (location: 3, time: 8783, ensemble: 11)
Coordinates:
  * location              (location) object 24B 'Escalade' ... 'Windy Point IIa'
    analog                (time, ensemble) datetime64[ns] 773kB NaT ... 1991-...
  * time                  (time) datetime64[ns] 70kB 2022-01-01T01:00:00 ... ...
  * ensemble              (ensemble) int32 44B 0 1 2 3 4 5 6 7 8 9 10
    lat                   (location) float64 24B 33.74 40.45 45.71
    lon                   (location) float64 24B -99.65 -88.7 -120.8
    forecast_date         datetime64[ns] 8B 2022-01-01
Data variables:
    hub_height            (location) int64 24B 119 80 80
    elevation             (location) int64 24B 444 260 582
    rated_capacity        (location) float64 24B 5.6 1.65 2.3
    turbine_count         (location) int64 24B 65 121 24
    turbine_model         (location) object 24B 'V162-5.6' ... 'SWT-2.3-93'
    turbine_manufacturer  (location) object 24B 'Vestas' 'Vesta

## Wind to Power

Now that we have wind and temperature, let's translate them to megawatts.


### Shear to Hub Height

Each of the sites has a different tower height. We need to interpolate or extrapolate 10m and 100m wind speeds to the hub height at each location.


In [ ]:
hst["wspdhh"] = sk.wind.shear_wind(hst["wspd100"], hst["wspd"], hst["hub_height"])
print(hst)

if loc.location_file:
    hst.plot.scatter(x="wspd100", y="wspdhh", hue="location")
else:
    hst[["wspd100", "wspdhh"]].mean(dim="time").plot.scatter(x="wspd100", y="wspdhh")
    plt.title("Shear to Hub Height")

plt.autoscale(False)
plt_diag = plt.plot([0, 25], [0, 25], "--k")

<xarray.Dataset> Size: 909kB
Dimensions:               (location: 3, time: 8737)
Coordinates:
  * location              (location) object 24B 'Escalade' ... 'Windy Point IIa'
  * time                  (time) datetime64[ns] 70kB 2022-01-01 ... 2022-12-31
    lat                   (location) float64 24B 33.74 40.45 45.71
    lon                   (location) float64 24B -99.65 -88.7 -120.8
Data variables:
    hub_height            (location) int64 24B 119 80 80
    elevation             (location) int64 24B 444 260 582
    rated_capacity        (location) float64 24B 5.6 1.65 2.3
    turbine_count         (location) int64 24B 65 121 24
    turbine_model         (location) object 24B 'V162-5.6' ... 'SWT-2.3-93'
    turbine_manufacturer  (location) object 24B 'Vestas' 'Vestas' 'Siemens'
    year_online           (location) int64 24B 2022 2007 2009
    wspd                  (time, location) float64 210kB 3.592 3.517 ... 1.135
    wspd100               (time, location) float64 210kB 6.842 8.0

### Correct for Air Density

Manufacturers denominate their power curves in terms of air density, often sea level. We will use elevation and temperature to modify wind speeds to match the power curve.


In [ ]:
hst["wspddc"] = sk.wind.correct_wind_density(
    wspd=hst["wspdhh"],
    dens=1.225,
    temp=hst["temp"],
    elev=hst["elevation"] + hst["hub_height"],
)

if loc.location_file:
    with xr.Dataset({"temp": hst.temp, "dif": hst.wspddc / hst.wspdhh}) as dif:
        dif.plot.scatter(x="temp", y="dif", hue="location")
elif loc.shapefile:
    with xr.Dataset(
        {"temp": hst.temp, "elevation": hst.elevation, "dif": hst.wspddc / hst.wspdhh}
    ).mean(dim="time") as dif:
        print(dif)
        dif.plot.scatter(x="temp", y="dif", hue="elevation", cmap="terrain", vmin=0)
plt.title("Wind Density Correction")
plt.ylabel("Corrected Wind Speed / Raw Wind Speed")
plt.xlabel("Temperature (K)");

### Define Turbine Power Curves


In [ ]:
pc = sk.wind.get_power_curve(turbine_model=hst["turbine_model"])
print(pc.head())
plt_curves = pc.plot.line(x="wind_speed")

<xarray.DataArray 'power_curve' (wind_speed: 5, turbine_model: 3)> Size: 120B
array([[0.   , 0.   , 0.   ],
       [0.027, 0.   , 0.   ],
       [0.144, 0.01 , 0.026],
       [0.289, 0.028, 0.053],
       [0.464, 0.075, 0.1  ]])
Coordinates:
  * wind_speed     (wind_speed) float64 40B 0.0 3.0 3.5 4.0 4.5
  * turbine_model  (turbine_model) object 24B 'V162-5.6' 'V82-1.65' 'SWT-2.3-93'
Attributes:
    units:          MW
    long_name:      Power
    standard_name:  power_curve


### Apply Power Curve

Feed the density-corrected hub-height wind speeds through the power curve to get power production.


In [ ]:
hst["power"] = sk.wind.calc_wind_power(hst["wspdhh"], hst["turbine_model"])
print(hst.power)

plot_timeseries(hst["power"])

<xarray.DataArray 'power' (time: 8737, location: 3)> Size: 210kB
array([[2.1645533 , 0.62240475, 0.        ],
       [2.96460284, 0.47617704, 0.        ],
       [3.32678462, 0.55230746, 0.        ],
       ...,
       [0.30359419, 0.        , 0.        ],
       [0.77323862, 0.00364891, 0.        ],
       [1.9277592 , 0.02549899, 0.        ]])
Coordinates:
  * location  (location) <U15 180B 'Escalade' 'Twin Groves I' 'Windy Point IIa'
  * time      (time) datetime64[ns] 70kB 2022-01-01 ... 2022-12-31
    lat       (location) float64 24B 33.74 40.45 45.71
    lon       (location) float64 24B -99.65 -88.7 -120.8
Attributes:
    units:          MW
    long_name:      Wind Power
    standard_name:  power


### End-to-End Wind-to-Power

As a convenience, the function `calc_wind_power_all` will shear, density correct, and calculate power. Let's use it with the `downscale` dataset so we don't have to run each step individually.


In [ ]:
pwr_dsc = sk.wind.calc_wind_power_all(dsc)
print(pwr_dsc)
plot_timeseries(pwr_dsc["power"])

<xarray.Dataset> Size: 5MB
Dimensions:        (location: 3, time: 8783, ensemble: 11)
Coordinates:
  * location       (location) object 24B 'Escalade' ... 'Windy Point IIa'
    analog         (time, ensemble) datetime64[ns] 773kB NaT ... 1991-12-17T2...
  * time           (time) datetime64[ns] 70kB 2022-01-01T01:00:00 ... 2023-01...
  * ensemble       (ensemble) int32 44B 0 1 2 3 4 5 6 7 8 9 10
    lat            (location) float64 24B 33.74 40.45 45.71
    lon            (location) float64 24B -99.65 -88.7 -120.8
    forecast_date  datetime64[ns] 8B 2022-01-01
Data variables:
    wspdhh         (time, location, ensemble) float64 2MB nan nan ... 2.217
    power          (time, location, ensemble) float64 2MB nan nan ... 0.638 0.0
